# Empirical Risk Minimization

## Risk minimization

Recall that our goal is to find a predictor $f: \mathcal{X} \rightarrow \mathcal{Y}$ such that $f(X)$ accurately predicts $Y$. To measure the level of accuray, we introduce a loss function $\ell: \mathcal{Y}\times \mathcal{Y}\rightarrow \mathbb{R}_{\geq 0}$. The value $\ell(f(X), Y)$ is large if $f$ is a poor predictor function. In practice, many loss functions are used depending on the context, for example

1. Square loss: $\ell(y, z)=(y-z)^2$
2. Zero-one loss: $\ell(y, z)= \mathbb{1}\{y\neq z\}$
3. Absolute loss: $\ell(y, z)=|y-z|$

We want to find a predictor such that $\ell(f(X), Y)$ is small on average. This can be framed as an optimization problem:

$$f^* = \underset{f}{\text{argmin}}\; \underbrace{\mathbb{E}_{(X, Y)\sim \mathcal{X}\times \mathcal{Y}}[\ell(f(X), Y)]}_{R(f)} \tag{1}$$

Note that expectation is included since $X, Y$ are random variables. The term $R(f)$ denote the risk, or the expected loss, of the predictor $f$. The solution to $f^*$ is called the Bayes optimal predictor. For certain loss functions, $f^*$ can be determined directly <br>

<b>Example 1 (Square loss function) </b>
When $\ell(y, z)=(y-z)^2$, the Bayes optimal predictor is given by

$$ f^*(x)=\mathbb{E}[Y|X=x]$$

<i>Proof.</i> <br>
Instead of solving $(1)$ directly, note that we can solve it in a point-wise manner

$$
\begin{align*}
f^*(x) &= \underset{f}{\text{argmin}}\; \mathbb{E}[(f(X)- Y)^2|X=x]\\
&= \underset{z\in \mathcal{Y}}{\text{argmin}}\; \mathbb{E}[(z-Y)^2|X=x]
\end{align*}
$$

The objective can be written as 

$$
\begin{align*}
\mathbb{E}[(z-Y)^2|X=x]
&= \mathbb{E}[(z- \mathbb{E}[Y|X=x] + \mathbb{E}[Y|X=x] - Y)^2|X=x]\\
&= \mathbb{E}[(z- \mathbb{E}[Y|X=x])^2|X=x] + 2\mathbb{E}[(z- \mathbb{E}[Y|X=x])(\mathbb{E}[Y|X=x]-Y)|X=x] + \mathbb{E}[(\mathbb{E}[Y|X=x]-Y)^2|X=x]\\
&= \mathbb{E}[(z- \mathbb{E}[Y|X=x])^2|X=x] + \mathbb{E}[(\mathbb{E}[Y|X=x]-Y)^2|X=x]\\
\end{align*}
$$

Since the last term in independent of $z$, it follows that the minimizer is given by $z=\mathbb{E}[Y|X=x]$. This proves the claim. This example tells us that for regression problems, the Bayes optimal predictor is simply given by the conditional expectation. 

Despite $(1)$ can be solved for certain loss functions, in general, solving $(1)$ is extremly challenging, for three main reasons

1. The feature/response distributions are typically unknown.
2. Even when the distributions are known, solving $(1)$ involves minimizing a function within an integral. And unfortunately, there aren't efficient numerical algorithms available for solving these types of problems.
3. The space of all feasible functions is a large set.

This means that in practice, we often cannot solve $(1)$ exactly. Therefore, we need to find ways to simplify the optimization problem.

## Improvement 1: Empirical risk minimization

One crucial observation is that while the distributions are often unknown in practice, we do have access to observations $\mathcal{D}=\{(X_i, y_i)\}_{i\in [n]}$ sampled from the original distribution. This is important because, under certain regularity conditions, the law of large numbers suggests that

$$\frac{1}{n}\sum_{i=1}^n \ell(f(X_i), y_i)\;\xrightarrow{\;\;n \to \infty\;\;}\;\mathbb{E}_{(X, Y)\sim \mathcal{X}\times \mathcal{Y}}[\ell(f(X), Y)]$$

This motivates us to replace the objective in $(1)$ as 

$$\hat{f} =  \underset{f}{\text{argmin}}\; \underbrace{\frac{1}{n}\sum_{i=1}^n \ell(f(X_i), y_i)}_{\hat{R}(f)} \tag{2}$$

Where the term $\hat{R}(f)$ is called the empirical risk, it is an estimate of the true risk. The optimization problem $(2)$ resolves point 1 and 2 presented earlier, however, the set of feasible function is still a large set. 

## Improvement 2: Constrained optimization

A straightforward method to address point 3 is by constraining the set of possible functions. Rather than optimizing over all measurable functions, we confine the feasible function to the set $\mathcal{F}$. This gives us the following

$$\hat{f}_{\mathcal{F}} = \underset{f\in \mathcal{F}}{\text{argmin}}\; \frac{1}{n}\sum_{i=1}^n \ell(f(X_i), y_i) \tag{3}$$

Optimization problem $(3)$ is in general solvable. If the functions in $\mathcal{F}$ is parameterized by $\theta\in \Theta$, then $(3)$ is equivalent to a parameter estimation problem

$$\hat{\theta} = \underset{\theta \in \Theta}{\text{argmin}}\; \frac{1}{n}\sum_{i=1}^n \ell(f_{\theta}(X_i), y_i)$$

Which can be solved using algorithms like stochastic gradient descent. This gives us a parametric model. If, on the other hand, $\mathcal{F}$ is not parameterized, then by choosing a good set of functions, in general one can solve $(3)$ using other methods. This gives us a nonparametric model. 

## Example: Linear regression

Suppose we restrict ourselves on the set of linear functions $\mathcal{F}=\{f:f(x)=\theta^Tx\}$. Given observations $\mathcal{D}=\{(X_i, y_i)\}_{i\in [n]}$, and using the square loss, the optimal parameter is given by 

$$\hat{\theta} = \underset{\theta}{\text{argmin}}\; \frac{1}{n}\sum_{i=1}^n (\theta^TX_i- y_i)^2$$

Letting $X, y$ denote the matrix of features/response, the above is equivalent to 

$$\hat{\theta} = \underset{\theta}{\text{argmin}}\; \frac{1}{n}||X\theta-y||_2^2$$

This optimization problem can be solved using standard first order methods. From the first order condition

$$\nabla_{\theta} = 2(X^TX)\theta - 2(X^Ty) = 0\implies \hat{\theta} = (X^TX)^{-1}(X^Ty)$$

And this gives us the optimal predictor under empirical risk minimization. 